In [9]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv')
melb_data.head(3)
melb_data.loc[[0, 1, 2],['Address', 'Price']]

,Address,Price
0,85 Turner St,1480000.0
1,25 Bloomburg St,1035000.0
2,5 Charles St,1465000.0


In [11]:
melb_df = melb_data.copy()

In [13]:
melb_df.drop(['index', 'Coordinates'], axis=1, inplace=True)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [15]:
total_rooms = melb_df['Rooms']+melb_df['Bathroom']+melb_df['Bedroom']
#total_rooms
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
melb_df['MeanRoomsSquare']

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [16]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
melb_df['AreaRatio']

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

In [19]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
sitizen_per_km = countries_df['population']*1e6 / countries_df['square']
print(round(sitizen_per_km.mean(),2))

84.93


In [23]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['Date'].head()
years_sold = melb_df['Date'].dt.year
print(years_sold.head())
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0    2016
1    2016
2    2017
3    2017
4    2016
Name: Date, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [24]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

In [27]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
delta_days.dt.days.head()

0    337
1     34
2    428
3    428
4    155
Name: Date, dtype: int64

In [32]:
#melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
print(melb_df['AgeBuilding'].head())
#melb_df = melb_df.drop('YearBuilt', axis=1)

0     46
1    116
2    117
3     47
4      2
Name: AgeBuilding, dtype: int64


In [34]:
# practice
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[melb_df['WeekdaySale']>4].shape[0]
print(weekend_count)

12822
